In [141]:
from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, G2, pair, GT
from charm.core.engine.util import objectToBytes
from charm.toolbox.IBSig import IBSig
from charm.schemes.pksig.pksig_bls04 import BLS01
import sys,random

In [142]:
class BLS01Aggregation(BLS01):
    def __init__(self, groupObj):
        super().__init__(groupObj)  
        self.group = groupObj  
    def dump(self, obj):
        return objectToBytes(obj, self.group)

    def keygen(self, secparam=None, g=None):
        if g is None:
            g = self.group.random(G2)
        x = self.group.random()
        g_x = g ** x
        pk = {'g^x': g_x, 'g': g, 'identity': str(g_x), 'secparam': secparam}
        sk = {'x': x}
        return (pk, sk)

    def sign(self, x, message):
        M = self.dump(message)
        return self.group.hash(M, G1) ** x

    def verify(self, pk, sig, message):
        M = self.dump(message)
        h = self.group.hash(M, G1)
        if pair(sig, pk['g']) == pair(h, pk['g^x']):
            return True  
        return False 
    

    def aggregation_verification(self, aggr_signature, publickey, msg, user_count , generator):
        combined_rhs = None
        lhs = pair(aggr_signature, generator)
        # print("\n",lhs)
        # rhs = {}
        for i in range(user_count):
            M = self.dump(msg[i])
            h = self.group.hash(M, G1)
            rhs = pair(h,publickey[i]['g^x'])
            if combined_rhs is None:
                combined_rhs = rhs
            else:
                combined_rhs = combined_rhs * rhs
        # print("rhs",combined_rhs)
        if lhs==combined_rhs:
            return True
        else:
            return False
    
    def aggregate_verify(self, signatures_agg, messages, public_keys, g2):
        """
        Verify an aggregate BLS signature.

        Args:
            signatures_agg: Aggregate signature in G1
            messages: List of distinct messages [m1, m2, ..., mk]
            public_keys: Dictionary of public keys {0: pk0, 1: pk1, ..., k: pkk} where pk_i = {'g^x': v_i, ...}
            g2: Generator of G2

        Returns:
            bool: True if valid, False otherwise
        """
        # Step 1: Check that messages are distinct
        if len(messages) != len(set(messages)):
            return False

        # Step 2: Compute h_i = H(self.dump(m_i)) for each message
        h = [self.group.hash(self.dump(m), G1) for m in messages]

        # Step 3: Compute LHS = e(signatures_agg, g2)
        lhs = pair(signatures_agg, g2)

        # Step 4: Compute RHS = product of e(h_i, v_i)
        rhs = self.group.init(GT, 1)  # Identity element in GT
        for i in range(len(messages)):
            rhs *= pair(h[i], public_keys[i]['g^x'])

        # Step 5: Check equality
        return lhs == rhs
    

In [143]:
from charm.toolbox.pairinggroup import PairingGroup
group = PairingGroup('MNT224')
bls_aggregation = BLS01Aggregation(group)

messages = ['Tom friends with Charlie',
           'Where is the gold?',
           'Charlie said he found the gold',
           'Tom asked where?',
           'Charlie said in the Ground.',
           'Tom asking for a shovel.',
           'Charlie Does not have a shovel',
           'Tom Goes to Walmart and buys a shovel',
           'Charlie uses the shovel to dig the ground',
            'Tom and Charlie Finds Gold !'
           ]


users = [str(usr) for usr in range(10)]

'''
Message Association {User0: Msg0} ... {UserN: MsgN}
0:"Tom friends with Charlie"
'''

#############------Generate PublicKey and Private/SecretKeys With a Generator------########
'''
'''

user_secret_keys, user_public_keys = dict(), dict()
generator = ''

for index in range(0,len(users)):
    if index==0:
        pk, sk = bls_aggregation.keygen()
        user_secret_keys[index] = sk
        user_public_keys[index] = pk
        generator = pk['g']
    
    else:
        pk , sk = bls_aggregation.keygen(g=generator)
        user_secret_keys[index] = sk
        user_public_keys[index] = pk
        

#print('public:\n',user_public_keys)
#print('Secret:\n',user_secret_keys)

signature_dictionary = {}

for index in range(0,len(users)):
    sig = bls_aggregation.sign(user_secret_keys[index]['x'], messages[index])
    signature_dictionary[index] = sig

#print(signature_dictionary)




### Aggregate Signatures

#### Signatures from distinct Users for distinct messages can be aggregated. This Process is also known as Short Signature


In [144]:
###################-----Aggregate the Signatures Dictionary--------################

SIGNATURE_AGGREGATE = None
for key,sign in signature_dictionary.items():
    if SIGNATURE_AGGREGATE is None:
        SIGNATURE_AGGREGATE = sign
    else:
        SIGNATURE_AGGREGATE = SIGNATURE_AGGREGATE + sign 

# #print('SIGNATURE_AGGREGATE ',SIGNATURE_AGGREGATE)
# SIGNATURE_AGGREGATE = bls_aggregation.aggregate_signatures(signature_dictionary)

In [145]:
###############---------Verify the Aggregated Signature----------############


is_verified = bls_aggregation.aggregate_verify(SIGNATURE_AGGREGATE, messages, user_public_keys,generator)
                                              
print('Message Verification: ',is_verified)



Message Verification:  True


# Assignment 1 - Q1 

## Why for a valid aggregation verification for all the messages should be distinct ? 

### Solution

#### In the BLS signature scheme, multiple signatures from different users can be combined into a single aggregated signature. Each user signs its own message using their private key, and these signatures are multiplied together to form an aggregate. Verification  process then checks if this aggregated signature matches the expected combination of the messages and the users’ public keys. The  whole process is based on the mathematical properties of pairing-based alogrithms in cryptography. 

However it relies on one concept that all messages that are being signed are all distinct. 


#### The main reason for having distinct messages is to prevent attacks that could exploit the way signatures are combined.
#### When messages are not distinct (i.e., some users sign the same message), the aggregated signature can become ambiguous or manipulable, which allows an attacker to forge signatures or misrepresent who signed what ( An Identity Based Signature may solve this problem). 

###### One User may sign the message for another User but its identity will remain hidden as mathematically its not possible to know, who signed what in the current setting. 

##### Most Common Known Attack of this Kind is
#### Rogue Key Attack



#### By Keeping the messages distinct we will be able to keep the attack away from the system, however there are other techniques that can help in mitigating this issue. 

Full Mathematical Detail: 
` https://docs.google.com/document/d/1WcHOY1vJq5RDEiRBGRepwwPQViXTU6e7jy_olBGnhzI/edit?usp=sharing` 


##### Paper:
- Aggregate and Verifiably Encrypted Signatures from Bilinear Maps
- Paper Section 3.2 - Definition 5